# Specify options

You can use options to customize the Estimator and Sampler primitives. This section focuses on how to specify Qiskit Runtime primitive options. While the interface of the primitives' `run()`  method is common across all implementations, their options are not. Consult the corresponding API references for information about the [`qiskit.primitives`](/api/qiskit/primitives#primitives) and [`qiskit_aer.primitives`](https://qiskit.github.io/qiskit-aer/apidocs/aer_primitives.html) options.

Notes about specifying options in the primitives:

- `SamplerV2` and `EstimatorV2` have separate options classes. You can see the available options and update option values during or after primitive initialization.
- Use the `update()` method to apply changes to the `options` attribute.
- If you do not specify a value for an option, it is given a special value of `Unset` and the server defaults are used.
- The `options` attribute is the `dataclass` Python type.  You can use the built-in `asdict` method to convert it to a dictionary.

<span id="pass-options"></span>
## Set primitive options

You can set options when initializing the primitive, after initializing the primitive, or in the `run()` method. See the [precedence rules](runtime-options-overview#options-precedence) section to understand what happens when the same option is specified in multiple places.

### Primitive initialization

You can pass in an instance of the options class or a dictionary when initializing a primitive, which then makes a copy of those options. Thus, changing the original dictionary or options instance doesn't affect the options owned by the primitives.

#### Options class

When creating an instance of the `EstimatorV2` or `SamplerV2` class, you can pass in an instance of the options class. Those options will then be applied when you use `run()` to perform the calculation.  Specify the options in this format:  `options.option.sub-option.sub-sub-option = choice`.  For example: `options.dynamical_decoupling.enable = True`

Example:

`SamplerV2` and `EstimatorV2` have separate options classes ([`EstimatorOptions`](/api/qiskit-ibm-runtime/options-estimator-options) and [`SamplerOptions`](/api/qiskit-ibm-runtime/options-sampler-options)).

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.options import EstimatorOptions

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

options = EstimatorOptions(
    resilience_level=2,
    resilience={"zne_mitigation": True, "zne": {"noise_factors": [1, 3, 5]}},
)

# or...
options = EstimatorOptions()
options.resilience_level = 2
options.resilience.zne_mitigation = True
options.resilience.zne.noise_factors = [1, 3, 5]

estimator = Estimator(mode=backend, options=options)

#### Dictionary

You can specify options as a dictionary when initializing the primitive.

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Setting options during primitive initialization
estimator = Estimator(
    backend,
    options={
        "resilience_level": 2,
        "resilience": {
            "zne_mitigation": True,
            "zne": {"noise_factors": [1, 3, 5]},
        },
    },
)

### Update options after initialization

You can specify the options in this format: `primitive.options.option.sub-option.sub-sub-option = choice` to take advantage of auto-complete, or use the `update()` method to make bulk updates.

The `SamplerV2` and `EstimatorV2` options classes ([`EstimatorOptions`](/api/qiskit-ibm-runtime/options-estimator-options) and [`SamplerOptions`](/api/qiskit-ibm-runtime/options-sampler-options)) do not need to be instantiated if you are setting options after initializing the primitive.

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

estimator = Estimator(mode=backend)

# Setting options after primitive initialization
# This uses auto-complete.
estimator.options.default_shots = 4000
# This does bulk update.
estimator.options.update(
    default_shots=4000, resilience={"zne_mitigation": True}
)

<span id="run-method"></span>
### Run() method

The only values you can pass to `run()` are those defined in the interface.  That is, `shots` for Sampler and `precision` for Estimator. This overwrites any value set for `default_shots` or `default_precision` for the current run.

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit.library import random_iqp
from qiskit.transpiler import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

circuit1 = random_iqp(3)
circuit1.measure_all()
circuit2 = random_iqp(3)
circuit2.measure_all()

pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend
)

transpiled1 = pass_manager.run(circuit1)
transpiled2 = pass_manager.run(circuit2)

sampler = Sampler(mode=backend)
# Default shots to use if not specified in run()
sampler.options.default_shots = 500
# Sample two circuits at 128 shots each.
sampler.run([transpiled1, transpiled2], shots=128)

# Sample two circuits with different numbers of shots.
# 100 shots is used for transpiled1 and 200 for transpiled.
sampler.run([(transpiled1, None, 100), (transpiled2, None, 200)])

<RuntimeJobV2('cz1b4d910wx0008az1wg', 'sampler')>

### Special cases

#### Resilience level (Estimator only)

The resilience level is not actually an option that directly impacts the primitive query, but specifies a base set of curated options to build off of. In general, level 0 turns off all error mitigation, level 1 turns on options for measurement error mitigation, and level 2 turns on options for gate and measurement error mitigation.

Any options you manually specify in addition to the resilience level are applied on top of the base set of options defined by the resilience level. Therefore, in principle, you could set the resilience level to 1, but then turn off measurement mitigation, although this is not advised.

In the following example, setting the resilience level to 0 initially turns off `zne_mitigation`, but `estimator.options.resilience.zne_mitigation = True` overrides the relevant setup from `estimator.options.resilience_level = 0`.

In [5]:
from qiskit_ibm_runtime import EstimatorV2, QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

estimator = EstimatorV2(backend)

estimator.options.default_shots = 100
estimator.options.resilience_level = 0
estimator.options.resilience.zne_mitigation = True

#### Shots (Sampler only)

The `SamplerV2.run` method accepts two arguments: a list of PUBs, each of which can specify a PUB-specific value for shots, and a shots keyword argument. These shot values are a part of the Sampler execution interface, and are independent of the Runtime Sampler's options.  They take precedence over any values specified as options in order to comply with the Sampler abstraction.

However, if `shots` is not specified by any PUB or in the run keyword argument (or if they are all `None`), then the shots value from the options is used, most notably `default_shots`.

Finally, because the [`twirling` options](/api/qiskit-ibm-runtime/options-twirling-options) `num_randomizations` and `shots_per_randomization` are enabled by default, the number of shots will actually be the product of `num_randomizations` and `shots_per_randomization` if the `default_shots` value is the only way shots are specified.

To summarize, this is the order of precedence for specifying shots in the Sampler, for any particular PUB:

1. If the PUB specifies shots, use that value.
2. If the `shots` keyword argument is specified in `run`, use that value.
3. If `num_randomizations` and `shots_per_randomization` are specified as `twirling` options (enabled by default), shots are the product of those values.
3. If `sampler.options.default_shots` is specified, use that value.

Thus, if shots are specified in all possible places, the one with highest precedence (shots specified in the PUB) is used.

#### Precision (Estimator only)

Precision is analogous to shots, described in the previous section, except that the Estimator options contain both `default_shots` and `default_precision`.

Specifically, for any particular Estimator PUB:

1. If the PUB specifies precision, use that value.
2. If the precision keyword argument is specified in `run`, use that value.
2. If `num_randomizations` and `shots_per_randomization` are specified as [`twirling` options](/api/qiskit-ibm-runtime/options-twirling-options) (enabled by default), use their product to control the amount of data.
3. If `estimator.options.default_shots` is specified, use that value to control the amount of data.
4. If `estimator.options.default_precision` is specified, use that value.

For example, if precision is specified in all four places, the one with highest precedence (precision specified in the PUB) is used.

<Admonition type="note">
Precision scales inversely with usage.  That is, the lower the precision, the more QPU time it takes to run.
</Admonition>

## Commonly used options

There are many available options, but the following are the most commonly used:

<span id="shots"></span>
### Shots
For some algorithms, setting a specific number of shots is a core part of their routines.  Shots (or precision) can be specified in multiple places.  They are prioritized as follows:

For any Sampler PUB:

1. Integer-valued shots contained in the PUB
2. The `run(...,shots=val)` value
3. The `options.default_shots` value

For any Estimator PUB:

1. Float-valued precision contained in the PUB
2. The `run(...,precision=val)` value
3. The `options.default_shots` value
4. The `options.default_precision` value

Example:

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit.library import random_iqp
from qiskit.transpiler import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

circuit1 = random_iqp(3)
circuit1.measure_all()
circuit2 = random_iqp(3)
circuit2.measure_all()

pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend
)

transpiled1 = pass_manager.run(circuit1)
transpiled2 = pass_manager.run(circuit2)


# Setting shots during primitive initialization
sampler = Sampler(mode=backend, options={"default_shots": 4096})

# Setting options after primitive initialization
# This uses auto-complete.
sampler.options.default_shots = 2000

# This does bulk update.  The value for default_shots is overridden if you specify shots with run() or in the PUB.
sampler.options.update(
    default_shots=1024, dynamical_decoupling={"sequence_type": "XpXm"}
)

# Sample two circuits at 128 shots each.
sampler.run([transpiled1, transpiled2], shots=128)

<RuntimeJobV2('cz1b4xkkvm9g008fpca0', 'sampler')>

### Maximum execution time

The maximum execution time (`max_execution_time`) limits how long a job can run.  If a job exceeds this time limit, it is forcibly canceled.  This value applies to single jobs, whether they are run in job, session, or batch mode.

The value is set in seconds, based on quantum time (not wall clock time), which is the amount of time that the QPU is dedicated to processing your job.  It is ignored when using local testing mode because that mode does not use quantum time.

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

estimator = Estimator(mode=backend)

estimator.options.max_execution_time = 2500

<span id="no-error-mitigation"></span>
## Turn off all error mitigation and error suppression

You can turn off all error mitigation and suppression if you are, for example, doing research on your own mitigation techniques. To accomplish this, for EstimatorV2, set `resilience_level = 0`. For SamplerV2, no changes are necessary because no error mitigation or suppression options are enabled by default.

Example:

Turn off all error mitigation and suppression in Estimator.

In [8]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator, QiskitRuntimeService

# Define the service.  This allows you to access IBM QPU.
service = QiskitRuntimeService()

# Get a backend
backend = service.least_busy(operational=True, simulator=False)

# Define Estimator
estimator = Estimator(backend)

options = estimator.options

# Turn off all error mitigation and suppression
options.resilience_level = 0

## Next steps

<Admonition type="tip" title="Recommendations">
    - Find more details about the `EstimatorV2` methods in the [Estimator API reference](../api/qiskit-ibm-runtime/estimator-v2).
    - Find more details about the `SamplerV2` methods in the [Sampler API reference](../api/qiskit-ibm-runtime/sampler-v2).
    - Find details about how to configure [error suppression](configure-error-suppression) and [error mitigation](configure-error-mitigation).
    - Decide what [execution mode](execution-modes) to run your job in.
</Admonition>